Помимо стандартных HTML-тегов, шаблонизатор Django поддерживает [несколько десятков собственных тегов](https://docs.djangoproject.com/en/3.2/ref/templates/builtins/#ref-templates-builtins-tags). 

С помощью тегов шаблонизатора можно собирать шаблоны из отдельных фрагментов,  выполнять логические операции с переменными и делать множество других полезных вещей.

Теги шаблонизатора обозначаются так: `{% tag %}`. Для удобства восприятия удобно ставить пробел между именем тега и символами `%`. 

Теги бывают одиночные и парные. У парных есть начальные и конечные теги: `{% tag %} ... {% endtag %}`. 

***
## Избавляемся от дублирования кода: теги {% extends %} и {% block %}

Практически на любом сайте есть повторяющиеся фрагменты. Например, содержимое на всех страницах разное, а шапка и подвал остаются неизменными.

В HTML-коде страницы комментариями отмечены фрагменты, которые повторяются на всех страницах, и части, которые на каждой странице содержат уникальную информацию:

***
```html
<!DOCTYPE html>
<html lang="ru">
  <head>
  </head>
  <body>
    <!-- header повторяется на всех страницах -->
    <header>
      <p>Шапка страницы</p>
    </header>
    <main>
      <!-- Центральная часть - разная на всех страницах -->
      <p>Уникальное содержимое веб-страницы</p>
    </main>
    <!-- footer повторяется на всех страницах -->
    <footer>
      <p>Подвал</p>
    </footer>
  </body>
</html>
```
***

Если в проекте применяется десять шаблонов, то разработчику нужно будет десять раз вставить одинаковое содержимое `footer` и `header` во все шаблоны. А потом ещё десять раз исправить их, если придётся что-нибудь изменить в шапке или в подвале. Так, конечно, не стоит делать.

Для решения этой задачи в Django есть теги `{% extends %}` (англ. extend, «расширять») и `{% block %}` (англ. block, «блок»).

Вот алгоритм их использования: 

1. Разработчик создаёт **базовый шаблон** (обычно его называют base.html), он содержит тот HTML-код, который повторяется на всех страницах сайта. Это своего рода «рамка», обрамление страницы.

2. В базовом шаблоне размечаются области, куда можно вставить уникальное для каждой страницы содержимое. Для разметки таких «посадочных мест» применяют парный тег `{% block имя_блока %} — {% endblock %}`. 
На одной странице может быть несколько разных блоков, имя каждого блока должно быть уникальным в пределах шаблона.

***
```html
 <!-- base.html -->
 <!DOCTYPE html>
 <html lang="ru">
   <head>
   </head>
   <body>
     <header>
       <p>Шапка страницы</p>
     </header>
     <main>
       <!-- Изменяемая часть: разная на всех страницах -->
       {% block content %}Контента пока нет :({% endblock %}
     </main>
     <footer>
       <p>Подвал</p>
     </footer>
   </body>
 </html>
```
***

  3. Разработчик создаёт **дочерние шаблоны**, в которых хранится лишь фрагмент HTML-кода — то самое уникальное содержимое страницы. Это содержимое тоже обрамлено тегами `{% block имя_блока %}` — `{% endblock %}`. Имена блоков должны совпадать с именами в *base.html*.

  Именно дочерний шаблон нужно вызывать из view-функции. При рендеринге страницы содержимое блоков дочернего шаблона будет вставлено в одноимённые блоки базового. 

  Чтобы Django мог определить, какой шаблон служит базовым, в начало кода дочерних шаблонов вставляют тег `{% extends "адрес_базового_шаблона" %}`.

  Напишем дочерний шаблон, в нём будет содержимое главной страницы:

***
  ```html
  <!-- index.html --> 
  {% extends "base.html" %}
  <!-- Имя блока — content, такое же, как в базовом шаблоне --> 
  {% block content %}
    <h1>Это главная страница проекта "ACME"!</h1>
  {% endblock %}
  ```
***

  Точно так же создаём второй дочерний шаблон, для страницы со списком товаров в каталоге:

***
```html
   <!-- product_list.html --> 
 {% extends "base.html" %}
 {% block content %}
   <h1>Это страница "ACME" со списком товаров!</h1>
 {% endblock %}
```
***

Теперь:

1. View-функция вызовет шаблон *product_list.html*.

2. Шаблонизатор увидит в коде файла *product_list.html* тег `{% extends "base.html" %}`.

3. Шаблонизатор вызовет файл *base.html* и вставит содержимое блоков дочернего файла в одноимённые блоки базового файла.

Готово, страница собрана!

![alt text](https://pictures.s3.yandex.net/resources/Image_-_2023-05-11T080805.004_1685546359.png)


Если же потребуется внести изменения в шапку или в подвал сайта, не будет нужды перебирать десятки файлов, внося одинаковые изменения в повторяющиеся фрагменты страниц. Надо только внести изменения в базовый шаблон — и они отобразятся на всех страницах, которые создаются на его основе. 

Поскольку базовый шаблон относится ко всем шаблонам проекта, его принято размещать прямо в корне папки `templates/`.

***
```
...
├── templates/
|   ├── catalog/           <-- Шаблоны приложения catalog
|   ├── homepage/          <-- Шаблоны приложения homepage
│   └── base.html          <-- Базовый шаблон
...
```

***
## Повторное использование HTML-кода. Тег {% include %}

HTML-документ — это, как правило, сотни строк кода. Чтобы упростить работу, шаблоны разделяют на части. Это позволит редактировать каждую часть отдельно, работая с небольшими фрагментами кода.

Хорошим решением будет отделить от базового шаблона код шапки сайта и код нижней части страницы (подвала) — и вынести эти фрагменты в отдельные файлы.

Отделённые фрагменты включаются в основной файл с помощью тега `{% include "адрес_подключаемого_файла" %}`.

При вызове основного файла шаблонизатор увидит тег `{% include %}` и вставит код из подключаемого файла на место этого тега.

![alt text](https://pictures.s3.yandex.net/resources/19_1676932845.png)

![alt text](https://pictures.s3.yandex.net/resources/20_1676932854.png)


Разделим файл base.html на части: вынесем в отдельные файлы шапку и подвал.

Создаём отдельные HTML-файлы:


*header.html*
***
```html
<!-- header.html -->
<header>
  <p>Шапка страницы</p>
</header>
```

*footer.html*

```html
<!-- footer.html -->
<footer>
  <p>Подвал</p>
</footer>
```
***

Подключаем их в шаблон base.html с помощью тега `{% include %}`. Аргументом тега должен быть адрес шаблона, код которого надо включить в файл, — `{% include "адрес_файла" %}`:


*base.html*
***
```html
<!-- base.html -->
<!DOCTYPE html>
<html lang="ru">
  <head>
  </head>
  <body>
    {% include "header.html" %}
    <main>
      {% block content %}Контента пока нет :({% endblock %}
    </main>
    {% include "footer.html" %}
  </body>
</html>
```
***

Шаблоны, встраиваемые через тег `{% include %}`, обычно хранят в директории *includes/*, вложенную в папку *templates/*.
***
```
...
├── templates/
|   ├── catalog/           <-- Шаблоны приложения catalog
|   ├── homepage/          <-- Шаблоны приложения homepage
│   ├── includes/          <-- Папка для повторяющихся частей шаблонов проекта
│   │   ├── footer.html
│   │   └── header.html
│   └── base.html          <-- Базовый шаблон
...
```
***

При таком размещении файлов к путям в `{% include %}` надо добавить имя директории:
***
```html
<!DOCTYPE html>
<html lang="ru">
  <head>
  </head>
  <body>
    {% include "includes/header.html" %}
    <main>
      {% block content %}Контента пока нет :({% endblock %}
    </main>
    {% include "includes/footer.html" %}
  </body>
</html>
```
***

Формальных требований к тому, как именно разбивать шаблоны, нет. Будет здорово, если код отдельных шаблонов не будет превышать 200–300 строк. 

Свои директории includes/ могут быть и внутри приложений. Например, в интернет-каталоге списки товаров зачастую отображаются в виде карточек: одна карточка — один товар. По вёрстке это совершенно одинаковые элементы (различается только содержимое — картинка, название, цена), а на странице может быть несколько десятков таких карточек. 

Будет логично вынести код карточки в отдельный шаблон — и этот код можно будет встроить и в страницу каталога, и в страницу оформления заказа, и в какой-нибудь промоблок.

Основное правило: если какой-то блок кода используется больше двух раз — лучше вынести его в отдельный файл.

***
```
...

├── templates/   
│   ├── catalog/
│   │   ├── includes/          
│   │   │   └── card.html         <-- Карточка товара
│   │   ├── product_detail.html   
│   │   └── product_list.html
|   ├── homepage/                 <-- Шаблоны приложения homepage 
│   │   └── ...                  
│   ├── includes/          
│   │   ├── footer.html           <-- Нижняя часть страниц     
│   │   └── header.html           <-- Верхняя часть страниц      
│   └── base.html                 <-- Базовый шаблон       
...
```
***

Использование такой структуры показывает, кто на ком стоит и что к чему относится. Но эта структура — не догма: разные разработчики и разные команды сами решают, по какой системе разбивать и хранить шаблоны.